In [ ]:
import json
import os

from SPARQLWrapper import JSON, SPARQLWrapper

data_dir = "../data/datasets/ComplexWebQuestions"
input_path = os.path.join(data_dir, "ComplexWebQuestions_test.json")
output_path = os.path.join(data_dir, "ComplexWebQuestions_test_wans.json")

if os.path.exists(output_path):
    raise SystemExit("Output file already exists.")

sparql = SPARQLWrapper("http://localhost:3001/sparql")
sparql.setReturnFormat(JSON)

with open(input_path) as f:
    dataset = json.load(f)

with open(output_path, "w") as f:
    for json_obj in dataset:
        id = json_obj["ID"]
        question = json_obj["question"]

        answer_list = []

        query = json_obj["sparql"]
        sparql.setQuery(query)
        results = sparql.query().convert()
        for result in results["results"]["bindings"]:
            ans_id = result["x"]["value"].replace("http://rdf.freebase.com/ns/", "")
            query = f"""
                    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
                    PREFIX : <http://rdf.freebase.com/ns/>
                    SELECT distinct ?name WHERE {{
                        :{ans_id} :type.object.name ?name .
                        FILTER(LANGMATCHES(LANG(?name), 'en'))
                    }}
            """

            sparql.setQuery(query)
            results = sparql.query().convert()
            if results["results"]["bindings"]:
                ans_name = results["results"]["bindings"][0]["name"]["value"]
                answer_list.append({"kb_id": ans_id, "text": ans_name})

        data = {
            "ID": id,
            "question": question,
            "sparql": json_obj["sparql"],
            "answers": answer_list,
        }
        f.write(json.dumps(data) + "\n")